In [2]:
import pandas as pd
import numpy as np

#Paso Inicial
data = pd.read_csv('europe.csv')
k = 5
p = len(data)
n = len(data.columns)

def set_init_weights():
  w = []
  for _ in range(1,k*k):
    wi = np.random.rand(n)
    w.append(wi)
  return np.array(w)

def kohonen(init_radius = 1, init_learn_rate = 0.5, max_epochs = 1000):
  #Paso Inicial
  weights = set_init_weights()
  radius = init_radius
  learn_rate = init_learn_rate
  t = 1
  cut = False
  #Paso t
  while t < max_epochs and not cut:
    #Selecciono un registro de entrada Xp
    x_index = np.random.choice(data.index)
    x = data.values[x_index]
    print(x)
    #w_k = 
    t += 1
kohonen()

['Latvia' 64589 16800 4.4 72.93 1.1 -0.6 12.8]
['Poland' 312685 20200 4.2 76.25 1.9 -0.08 12.4]
['Greece' 131957 26300 3.3 80.05 4.3 0.06 17.4]
['Italy' 301340 30500 2.9 81.86 1.8 0.38 8.4]
['Netherlands' 41543 42000 2.3 80.91 1.6 0.45 4.4]
['Ukraine' 603550 7200 8.0 68.74 1.4 -0.63 7.9]
['Spain' 505370 30500 3.1 81.27 1.2 0.65 21.7]
['Ireland' 70273 40800 2.6 80.32 0.9 1.11 14.4]
['Belgium' 30528 37800 3.5 79.65 1.3 0.06 7.2]
['Luxembourg' 2586 80600 3.4 79.75 0.9 1.14 5.7]
['Sweden' 450295 40700 3.0 81.18 1.5 0.17 7.5]
['Bulgaria' 110879 13800 4.2 73.84 2.6 -0.8 9.6]
['Portugal' 92090 23400 3.7 78.7 2.3 0.18 12.7]
['United Kingdom' 243610 36500 4.5 80.17 2.7 0.55 8.1]
['Slovenia' 20273 28800 1.8 77.48 1.7 -0.19 11.8]
['Ukraine' 603550 7200 8.0 68.74 1.4 -0.63 7.9]
['Spain' 505370 30500 3.1 81.27 1.2 0.65 21.7]
['Belgium' 30528 37800 3.5 79.65 1.3 0.06 7.2]
['Sweden' 450295 40700 3.0 81.18 1.5 0.17 7.5]
['Finland' 338145 36000 3.3 79.41 2.0 0.07 7.8]
['Luxembourg' 2586 80600 3.4 79.75